In [2]:
import pandas as pd
import numpy as np

In [36]:
csv_paths = [
    "../batches/Batch_284001_batch_results.csv",
    "../batches/Batch_284002_batch_results.csv",
    "../batches/Batch_284003_batch_results.csv",
    "../batches/Batch_284004_batch_results.csv"
]

In [ ]:
def parse_df(df):
    parsed_data = {
        "image_names": [],
        "descriptions": [],
        "questions": [],
        "answers": [],
        "bounding_boxes": []
    }
    
    for i in range(len(df)):
        # Extract entry
        row = df.iloc[i]
        
        # do not add if description is empty
        if pd.isna(row['Answer.Descriptive_description']):
            continue
        
        # Add entries to the dict
        parsed_data["image_names"].append(row['Input.image_url'])
        parsed_data["descriptions"].append(row['Answer.Descriptive_description'])
        parsed_data["bounding_boxes"].append(row['Answer.annotated_images.boundingBoxes'])
        parsed_data["questions"].append([])
        parsed_data["answers"].append([])
        questions = []
        answers = []
        for qno in range(1, 9):
            question_column = "Answer.Question {}".format(qno)
            
            if pd.isna(df.iloc[0][question_column]):
                continue
            
            answer_column = "Answer.Answer {}".format(qno)
            parsed_data["questions"][-1].append(row[question_column]) # Add question
            parsed_data["answers"][-1].append(row[answer_column]) # Add answer
    return parsed_data

In [44]:
def merge_batches(paths):
    parsed_data_frames = []
    for csv_path in paths:
        df = pd.read_csv(csv_path)
        parsed_dict = parse_df(df)
        parsed_df = pd.DataFrame.from_dict(parsed_dict)
        parsed_data_frames.append(parsed_df)
    return pd.concat(parsed_data_frames, ignore_index=True)

In [45]:
final_df = merge_batches(csv_paths)

In [49]:
final_df.head()

,image_names,descriptions,questions,answers,bounding_boxes
0,map_2_inside_env_001806.png,There is a partial view of the room from a door,"[What is present in the room?, Is there any vi...","[There is nothing present in the room,, There ...","[{""height"":422,""label"":""Door"",""left"":251,""top""..."
1,map_2_inside_env_001063.png,Room with furniture and a victim,"[Is there a painting, Does the room have any f...","[Yes there is a painting on the wall, Yes it d...","[{""height"":63,""label"":""Painting"",""left"":396,""t..."
2,map_1_inside_env_004857.png,There is a corridor with a door and a few wind...,"[Is the door open?, How many windows are there...","[Yes, the door is open., There are four window...","[{""height"":315,""label"":""Door"",""left"":400,""top""..."
3,map_2_inside_env_002306.png,A door in a corridor.,"[Is there fire in the corridor?, Is the door c...","[No, there is no fire in the corridor., Yes, t...","[{""height"":142,""label"":""Door"",""left"":213,""top""..."
4,map_1_inside_env_002893.png,Room with a bookshelf and a yellow victim,"[Is there a door, Can a lever be seen, Is ther...","[No door visible, No lever can be seen, No tor...","[{""height"":274,""label"":""Bookshelf"",""left"":0,""t..."


In [50]:
final_df.to_csv("../batches/combined.csv")